In [1]:
import pandas as pd
import re

In [2]:
results_data = pd.read_csv('results.csv')

results_data.head()

,Strategy,Ground Truth Binary,Ground Truth Source,Japicmp Binary,Japicmp Source,Revapi Binary,Revapi Source,Roseau Binary,Roseau Source
0,RemoveTypeR21,1,1,1,1,1,1,1,1
1,ChangeRecordComponent0ToLongFromRecordApiR21,1,1,1,1,1,1,1,1
2,ReduceR21VisibilityToPackagePrivate,1,1,1,1,1,1,1,1
3,ChangeRecordComponent0ToByteVarargsFromRecordA...,1,1,1,1,1,1,1,1
4,ChangeRecordComponent0ToLongVarargsFromRecordA...,1,1,1,1,1,1,1,1


In [3]:
GROUND_TRUTH_KEY = 'Ground Truth'
tools_analyzed = ['Japicmp', 'Revapi', 'Roseau']

GENERAL_BREAKING_SUFFIX = ' Global'
BINARY_BREAKING_SUFFIX = ' Binary'
SOURCE_BREAKING_SUFFIX = ' Source'

full_data = pd.DataFrame(columns=['bench'])
full_data['bench'] = results_data['Strategy']
full_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'] = results_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
full_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}'] = results_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
full_data[f'{GROUND_TRUTH_KEY}{GENERAL_BREAKING_SUFFIX}'] = full_data[f'{GROUND_TRUTH_KEY}{BINARY_BREAKING_SUFFIX}'] | full_data[f'{GROUND_TRUTH_KEY}{SOURCE_BREAKING_SUFFIX}']

for tool in tools_analyzed:
    full_data[f'{tool}{BINARY_BREAKING_SUFFIX}'] = results_data[f'{tool}{BINARY_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
    full_data[f'{tool}{SOURCE_BREAKING_SUFFIX}'] = results_data[f'{tool}{SOURCE_BREAKING_SUFFIX}'].map(lambda x: True if x == 1 else False)
    full_data[f'{tool}{GENERAL_BREAKING_SUFFIX}'] = full_data[f'{tool}{BINARY_BREAKING_SUFFIX}'] | full_data[f'{tool}{SOURCE_BREAKING_SUFFIX}']

full_data

,bench,Ground Truth Binary,Ground Truth Source,Ground Truth Global,Japicmp Binary,Japicmp Source,Japicmp Global,Revapi Binary,Revapi Source,Revapi Global,Roseau Binary,Roseau Source,Roseau Global
0,RemoveTypeR21,True,True,True,True,True,True,True,True,True,True,True,True
1,ChangeRecordComponent0ToLongFromRecordApiR21,True,True,True,True,True,True,True,True,True,True,True,True
2,ReduceR21VisibilityToPackagePrivate,True,True,True,True,True,True,True,True,True,True,True,True
3,ChangeRecordComponent0ToByteVarargsFromRecordA...,True,True,True,True,True,True,True,True,True,True,True,True
4,ChangeRecordComponent0ToLongVarargsFromRecordA...,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144030,AddParameterJavaLangIntegerArrArrayVarargsToCo...,True,False,True,True,True,True,True,True,True,True,True,True
144031,AddParameterJavaUtilDateArrArrayToConstructorC...,True,True,True,True,True,True,True,True,True,True,True,True
144032,AddParameterJavaSecuritySecureRandomArrArrayVa...,True,False,True,True,True,True,True,True,True,True,True,True
144033,AddExceptionJavaIoIOExceptionToConstructorC334...,False,True,True,False,True,True,False,True,True,False,True,True


In [4]:
STRATEGIES = [
    re.compile(r'^Add.+ModifierTo.+$'),
    re.compile(r'^AddException.+ToConstructor.+In.+$'),
    re.compile(r'^AddException.+ToMethod.+In.+$'),
    re.compile(r'^AddImplementedInterfaceToType.+$'),
    re.compile(r'^AddMethodToType.+$'),
    re.compile(r'^AddModifier.+ToField.+In.+$'),
    re.compile(r'^AddModifier.+ToMethod.+In.+$'),
    re.compile(r'^AddParameter.+ToConstructor.+In.+$'),
    re.compile(r'^AddParameter.+ToMethod.+In.+$'),
    re.compile(r'^AddRecordComponent.+ToRecord.+$'),
    re.compile(r'^AddSuperClassToClass.+$'),
    re.compile(r'^ChangeField.+In.+TypeTo.+$'),
    re.compile(r'^ChangeMethod.+In.+TypeTo.+$'),
    re.compile(r'^ChangeParameter.+To.+FromConstructor.+In.+$'),
    re.compile(r'^ChangeParameter.+To.+FromMethod.+In.+$'),
    re.compile(r'^ChangeRecordComponent.+To.+FromRecord.+$'),
    re.compile(r'^ReduceConstructor.+In.+VisibilityTo.+$'),
    re.compile(r'^ReduceField.+In.+VisibilityTo.+$'),
    re.compile(r'^ReduceMethod.+In.+VisibilityTo.+$'),
    re.compile(r'^Reduce.+VisibilityTo.+$'),
    re.compile(r'^Remove.+ModifierIn.+$'),
    re.compile(r'^RemoveConstructor.+In.+$'),
    re.compile(r'^RemoveEnumValue.+In.+$'),
    re.compile(r'^RemoveException.+FromConstructor.+In.+$'),
    re.compile(r'^RemoveException.+FromMethod.+In.+$'),
    re.compile(r'^RemoveField.+In.+$'),
    re.compile(r'^RemoveImplementedInterface.+FromType.+$'),
    re.compile(r'^RemoveMethod.+In.+$'),
    re.compile(r'^RemoveModifier.+ToField.+In.+$'),
    re.compile(r'^RemoveModifier.+ToMethod.+In.+$'),
    re.compile(r'^RemoveParameter.+FromConstructor.+In.+$'),
    re.compile(r'^RemoveParameter.+FromMethod.+In.+$'),
    re.compile(r'^RemoveRecordComponent.+FromRecord.+$'),
    re.compile(r'^RemoveSuperClassFromClass.+$'),
    re.compile(r'^RemoveType.+$'),
]

def get_strategies_from_benchs(benchs):
    unique_strategies = set()
    for bench in benchs:
        for strategy in STRATEGIES:
            if strategy.match(bench):
                unique_strategies.add(strategy.pattern)
                break

    return sorted(unique_strategies)


In [5]:
not_binary_breaking_for_ground_truth_but_breaking_for_all_tools = full_data.query('`Ground Truth Binary` == False and `Japicmp Binary` == True and `Revapi Binary` == True and `Roseau Binary` == True')
not_binary_breaking_for_ground_truth_but_breaking_for_all_tools = not_binary_breaking_for_ground_truth_but_breaking_for_all_tools.sort_values(by='bench')

print(len(not_binary_breaking_for_ground_truth_but_breaking_for_all_tools.index))

binary_benchs = not_binary_breaking_for_ground_truth_but_breaking_for_all_tools['bench'].tolist()
binary_strategies = get_strategies_from_benchs(binary_benchs)

print(binary_strategies)

not_binary_breaking_for_ground_truth_but_breaking_for_all_tools[['bench']]

9452
['^AddModifier.+ToField.+In.+$', '^AddModifier.+ToMethod.+In.+$', '^AddParameter.+ToConstructor.+In.+$', '^AddParameter.+ToMethod.+In.+$', '^ChangeField.+In.+TypeTo.+$', '^ChangeMethod.+In.+TypeTo.+$', '^ChangeParameter.+To.+FromConstructor.+In.+$', '^ChangeParameter.+To.+FromMethod.+In.+$', '^Reduce.+VisibilityTo.+$', '^ReduceConstructor.+In.+VisibilityTo.+$', '^ReduceField.+In.+VisibilityTo.+$', '^ReduceMethod.+In.+VisibilityTo.+$', '^RemoveConstructor.+In.+$', '^RemoveField.+In.+$', '^RemoveImplementedInterface.+FromType.+$', '^RemoveMethod.+In.+$', '^RemoveModifier.+ToField.+In.+$', '^RemoveModifier.+ToMethod.+In.+$', '^RemoveParameter.+FromMethod.+In.+$', '^RemoveSuperClassFromClass.+$', '^RemoveType.+$']


,bench
82151,AddModifierFinalToFieldF365InApiC9
82167,AddModifierFinalToFieldF366InApiC9
82183,AddModifierFinalToFieldF367InApiC9
82198,AddModifierFinalToFieldF368InApiC9
82215,AddModifierFinalToFieldF369InApiC9
...,...
136546,RemoveSuperClassFromClassC3321
136150,RemoveSuperClassFromClassC3322
137554,RemoveSuperClassFromClassC3323
137522,RemoveSuperClassFromClassC3324


In [6]:
not_source_breaking_for_ground_truth_but_breaking_for_all_tools = full_data.query('`Ground Truth Source` == False and `Japicmp Source` == True and `Revapi Source` == True and `Roseau Source` == True')
not_source_breaking_for_ground_truth_but_breaking_for_all_tools = not_source_breaking_for_ground_truth_but_breaking_for_all_tools.sort_values(by='bench')

print(len(not_source_breaking_for_ground_truth_but_breaking_for_all_tools.index))

source_benchs = not_source_breaking_for_ground_truth_but_breaking_for_all_tools['bench'].tolist()
source_strategies = get_strategies_from_benchs(source_benchs)

print(source_strategies)

not_source_breaking_for_ground_truth_but_breaking_for_all_tools[['bench']]

20017
['^AddException.+ToConstructor.+In.+$', '^AddException.+ToMethod.+In.+$', '^AddModifier.+ToField.+In.+$', '^AddParameter.+ToConstructor.+In.+$', '^AddParameter.+ToMethod.+In.+$', '^AddRecordComponent.+ToRecord.+$', '^ChangeField.+In.+TypeTo.+$', '^ChangeMethod.+In.+TypeTo.+$', '^ChangeParameter.+To.+FromConstructor.+In.+$', '^ChangeParameter.+To.+FromMethod.+In.+$', '^ChangeRecordComponent.+To.+FromRecord.+$', '^ReduceConstructor.+In.+VisibilityTo.+$', '^ReduceField.+In.+VisibilityTo.+$', '^ReduceMethod.+In.+VisibilityTo.+$', '^RemoveConstructor.+In.+$', '^RemoveException.+FromConstructor.+In.+$', '^RemoveException.+FromMethod.+In.+$', '^RemoveField.+In.+$', '^RemoveImplementedInterface.+FromType.+$', '^RemoveMethod.+In.+$', '^RemoveModifier.+ToField.+In.+$', '^RemoveModifier.+ToMethod.+In.+$', '^RemoveParameter.+FromMethod.+In.+$', '^RemoveSuperClassFromClass.+$']


,bench
98660,AddExceptionJavaIoIOExceptionToConstructorC9Ja...
98425,AddExceptionJavaIoIOExceptionToConstructorC9Ja...
53667,AddExceptionJavaIoIOExceptionToMethodM1269InApiC7
53677,AddExceptionJavaIoIOExceptionToMethodM1271Java...
53687,AddExceptionJavaIoIOExceptionToMethodM1273Java...
...,...
98971,RemoveSuperClassFromClassC3318
98716,RemoveSuperClassFromClassC3319
136795,RemoveSuperClassFromClassC3320
136150,RemoveSuperClassFromClassC3322


In [7]:
all_strategies = set()
all_strategies.update(binary_strategies)
all_strategies.update(source_strategies)
all_strategies = sorted(all_strategies)
print(len(all_strategies))
print(all_strategies)

27
['^AddException.+ToConstructor.+In.+$', '^AddException.+ToMethod.+In.+$', '^AddModifier.+ToField.+In.+$', '^AddModifier.+ToMethod.+In.+$', '^AddParameter.+ToConstructor.+In.+$', '^AddParameter.+ToMethod.+In.+$', '^AddRecordComponent.+ToRecord.+$', '^ChangeField.+In.+TypeTo.+$', '^ChangeMethod.+In.+TypeTo.+$', '^ChangeParameter.+To.+FromConstructor.+In.+$', '^ChangeParameter.+To.+FromMethod.+In.+$', '^ChangeRecordComponent.+To.+FromRecord.+$', '^Reduce.+VisibilityTo.+$', '^ReduceConstructor.+In.+VisibilityTo.+$', '^ReduceField.+In.+VisibilityTo.+$', '^ReduceMethod.+In.+VisibilityTo.+$', '^RemoveConstructor.+In.+$', '^RemoveException.+FromConstructor.+In.+$', '^RemoveException.+FromMethod.+In.+$', '^RemoveField.+In.+$', '^RemoveImplementedInterface.+FromType.+$', '^RemoveMethod.+In.+$', '^RemoveModifier.+ToField.+In.+$', '^RemoveModifier.+ToMethod.+In.+$', '^RemoveParameter.+FromMethod.+In.+$', '^RemoveSuperClassFromClass.+$', '^RemoveType.+$']


# Strategies not evaluated

In [8]:
errors_data = pd.read_csv('errors.csv')
errors_data.head()

,Strategy,Error
0,RemoveConstructorC5InApiC5,Failed to package new api to jar
1,ReduceConstructorC5InApiC5VisibilityToPrivate,Failed to package new api to jar
2,AddParameterLongToConstructorC5InApiC5,Failed to package new api to jar
3,AddParameterLongVarargsToConstructorC5InApiC5,Failed to package new api to jar
4,AddParameterByteToConstructorC5InApiC5,Failed to package new api to jar


In [17]:
def uniformize_strategy_name(strategy):
    for s in STRATEGIES:
        if s.match(strategy):
            return s.pattern

    return None

analyzed_strategies = results_data.copy()
analyzed_strategies['StrategyUniform'] = results_data['Strategy'].map(uniformize_strategy_name)
analyzed_strategies = analyzed_strategies.groupby('StrategyUniform').agg({ 'Strategy': 'count' }).reset_index().sort_values(by='Strategy', ascending=False)

errors_strategies = errors_data.copy()
errors_strategies['StrategyUniform'] = results_data['Strategy'].map(uniformize_strategy_name)
errors_strategies = errors_strategies.groupby('StrategyUniform').agg({ 'Strategy': 'count' }).reset_index().sort_values(by='Strategy', ascending=False)

not_analyzed_strategies = sorted(set([s.pattern for s in STRATEGIES]) - set(analyzed_strategies['StrategyUniform'].tolist()))
not_analyzed_and_not_in_errors_strategies = sorted(set(not_analyzed_strategies) - set(errors_strategies['StrategyUniform'].tolist()))
print('Number of strategies not analyzed and not in errors:', len(not_analyzed_and_not_in_errors_strategies))
print(not_analyzed_and_not_in_errors_strategies)

Number of strategies not analyzed and not in errors: 1
['^RemoveParameter.+FromConstructor.+In.+$']


In [18]:
analyzed_strategies

,StrategyUniform,Strategy
14,^ChangeParameter.+To.+FromMethod.+In.+$,43520
12,^ChangeMethod.+In.+TypeTo.+$,28715
8,^AddParameter.+ToMethod.+In.+$,28160
6,^AddModifier.+ToMethod.+In.+$,8664
19,^ReduceMethod.+In.+VisibilityTo.+$,8151
11,^ChangeField.+In.+TypeTo.+$,5760
29,^RemoveModifier.+ToMethod.+In.+$,3800
27,^RemoveMethod.+In.+$,2964
24,^RemoveException.+FromMethod.+In.+$,2445
30,^RemoveParameter.+FromMethod.+In.+$,2048


In [19]:
errors_strategies

,StrategyUniform,Strategy
14,^ChangeParameter.+To.+FromMethod.+In.+$,35760
8,^AddParameter.+ToMethod.+In.+$,23180
12,^ChangeMethod.+In.+TypeTo.+$,21971
19,^ReduceMethod.+In.+VisibilityTo.+$,6285
6,^AddModifier.+ToMethod.+In.+$,4703
11,^ChangeField.+In.+TypeTo.+$,3840
28,^RemoveModifier.+ToMethod.+In.+$,3045
26,^RemoveMethod.+In.+$,2076
29,^RemoveParameter.+FromMethod.+In.+$,1683
23,^RemoveException.+FromMethod.+In.+$,1404
